In [1]:
print("Um cuspidor esta nascendo!")

Um cuspidor esta nascendo!


In [2]:
!pip install gradio duckdb pandas --quiet


In [41]:
import duckdb
import pandas as pd
url = 'https://raw.githubusercontent.com/alura-cursos/llamaIndex_pandas_query/refs/heads/main/Dados/vendas.csv'
con = duckdb.connect(database="/tmp/data-spitter", read_only=False)
#con.sql(f'SELECT ID_compra FROM "{url}" LIMIT 5')
#df = pd.read_csv(url)
#con.register("temp_df", df)
#con.execute("CREATE OR REPLACE TABLE vendas AS SELECT * FROM temp_df")
con.execute(f"SELECT * FROM read_csv_auto('{url}', header=True) LIMIT 5").fetchdf()

# Testa a leitura
#print(con.execute("SELECT ID_compra  FROM vendas LIMIT 5").fetchdf())

,ID_compra,filial,cidade,tipo_cliente,genero,tipo_produto,preco_unitario,quantidade,imposto_5%,total,data,hora,forma_pagamento,avaliacao
0,750-67-8428,A,Santo André,Membro,Feminino,Saúde e Beleza,74.69,7,26.1415,548.9715,2024-01-05,13:08:00,Carteira Digital,9.1
1,226-31-3081,C,São Caetano,Normal,Feminino,Eletrônicos,15.28,5,3.8200,80.2200,2024-03-08,10:29:00,Dinheiro,9.6
2,631-41-3108,A,Santo André,Normal,Masculino,Casa,46.33,7,16.2155,340.5255,2024-03-03,13:23:00,Cartão de Crédito,7.4
3,123-19-1176,A,Santo André,Membro,Masculino,Saúde e Beleza,58.22,8,23.2880,489.0480,2024-01-27,20:33:00,Carteira Digital,8.4
4,373-73-7910,A,Santo André,Normal,Masculino,Esportes e Viagem,86.31,7,30.2085,634.3785,2024-02-08,10:37:00,Carteira Digital,5.3


In [23]:
con.execute("INSTALL ducklake;")
#con.execute("ATTACH 'ducklake:metadata.ducklake' AS my_ducklake (DATA_PATH '/tmp/dklq-data-spitter');")
con.execute("USE my_ducklake;")
#con.execute("CREATE OR REPLACE TABLE vendas_duck AS SELECT * FROM temp_df")
con.execute("SELECT * FROM vendas_duck").fetchall()

[('750-67-8428',
  'A',
  'Santo André',
  'Membro',
  'Feminino',
  'Saúde e Beleza',
  74.69,
  7,
  26.1415,
  548.9715,
  '2024-01-05',
  '13:08:00',
  'Carteira Digital',
  9.1),
 ('226-31-3081',
  'C',
  'São Caetano',
  'Normal',
  'Feminino',
  'Eletrônicos',
  15.28,
  5,
  3.82,
  80.22,
  '2024-03-08',
  '10:29:00',
  'Dinheiro',
  9.6),
 ('631-41-3108',
  'A',
  'Santo André',
  'Normal',
  'Masculino',
  'Casa',
  46.33,
  7,
  16.2155,
  340.5255,
  '2024-03-03',
  '13:23:00',
  'Cartão de Crédito',
  7.4),
 ('123-19-1176',
  'A',
  'Santo André',
  'Membro',
  'Masculino',
  'Saúde e Beleza',
  58.22,
  8,
  23.288,
  489.048,
  '2024-01-27',
  '20:33:00',
  'Carteira Digital',
  8.4),
 ('373-73-7910',
  'A',
  'Santo André',
  'Normal',
  'Masculino',
  'Esportes e Viagem',
  86.31,
  7,
  30.2085,
  634.3785,
  '2024-02-08',
  '10:37:00',
  'Carteira Digital',
  5.3),
 ('699-14-3026',
  'C',
  'São Caetano',
  'Normal',
  'Masculino',
  'Eletrônicos',
  85.39,
  7,
  2

In [42]:
#csv operations
import chardet
import duckdb
import csv

class CSVOperator():
    def __init__(self, file):
        self.df = pd.DataFrame()
        self.primary_key = ""
        self.cols = []
        self.con = duckdb.connect(database=':memory:')
        self.table_name = ""
        if file is not None:
            self.load_bronze(file)
        else:
            return "Envie um file CSV."

    def get_file_options(self, filename):
        options = {}
        with open(filename, 'rb') as f:
            sample = f.read(10000)
            sniffer = csv.Sniffer()
            dialect = sniffer.sniff(sample)
            options["delimiter"] = dialect.delimiter
            options["encoding"] = chardet.detect(sample)['encoding']
        return options

    def preview_bronze(self):
        preview_df = self.con.execute(f"""
          SELECT * FROM landing_temp LIMIT 10
        """).fetchdf()
        return preview_df

    def set_primary_key(self, column_name):
        if column_name in self.df.columns:
            self.primary_key = column_name

    def load_bronze(self,file):

        try:

          self.con.execute(f"""
              CREATE TABLE landing_temp AS
              SELECT * FROM read_csv_auto('{file.name}', header=True)
          """)
          self.cols = [row[0] for row in self.con.execute("DESCRIBE landing_temp").fetchall()]
          self.table_name = file.name

          #op = self.get_file_options(file.name)
          # self.df = pd.read_csv(file.name,
          #                      # delimiter=op["encoding"],
          #                       #encoding=op["encoding"],
          #                       skipinitialspace=True,
          #                       quotechar='"'
          #                       )


          #self.cols = list(self.df.columns)



          #return self.df
        except Exception as e:
          return str(e)



In [43]:
import gradio as gr

def run_bronze(file):
    csv_operator = CSVOperator(file)
    return csv_operator

def preview_bronze(csv_operator):
    return csv_operator.preview_bronze()

def set_pk_column(csv_operator, pk_column):
    if csv_operator is not None:
        csv_operator.set_primary_key(pk_column)
    return csv_operator

#button options

def toggle_button_pk(csv_operator,checkbox_value):
    if checkbox_value:
        return gr.update(interactive=True, visible=True,choices=csv_operator.cols)
    else:
        return gr.update(interactive=False, visible=False)

with gr.Blocks(title="Arquitetura Medalhão - Data Spitter - CSV - multi Reads") as demo:
    gr.Markdown("#Arquitetura Medalhão (Bronze / Silver / Gold)")
    gr.Markdown("# Spitter Data Lakehouse Explorer")
    gr.Markdown("Leia seus dados, configure as operacoes, e maos a obra. Apresentamos suporte a cargas incrementais e de sobrescrita")


    #BronzeLayer
    with gr.Tab("Bronze Layer"):
        gr.Markdown("### Upload dos dados brutos")
        state_csv_operator = gr.State(None)
        state_choices = gr.State([])
        csv_bronze = gr.File(label="Selecione um CSV",file_count="single", type="filepath")
        csv_bronze.upload(run_bronze, inputs=csv_bronze, outputs=state_csv_operator)

        gr.Markdown("### Preview dos dados brutos")
        bronze_btn = gr.Button("Preview Bronze")
        bronze_table = gr.DataFrame(label="Bronze Data")
        bronze_btn.click(preview_bronze, inputs=state_csv_operator, outputs=bronze_table)

        pk_checkbox = gr.Checkbox(label="Does the table have a primary key??", value=False)
        pk_dropdown = gr.Dropdown(label="Select the primary key", choices=[], interactive=False, visible=False)
        pk_checkbox.change(toggle_button_pk, inputs=[state_csv_operator, pk_checkbox], outputs=pk_dropdown)
        pk_dropdown.change(set_pk_column, inputs=[state_csv_operator, pk_dropdown], outputs=state_csv_operator)


        bronze_btn = gr.Button("Gerar tabela Bronze")





    # # Camada Silver
    # with gr.Tab("Camada Silver"):
    #     gr.Markdown("### Limpeza e padronização dos dados")
    #     botao_silver = gr.Button("Transformar Bronze → Silver")
    #     silver_tabela = gr.DataFrame(label="Prévia dos dados Silver")
    #     botao_silver.click(transformar_para_silver, outputs=silver_tabela)

    # # Camada Gold
    # with gr.Tab("Camada Gold"):
    #     gr.Markdown("### Agregações e métricas finais")
    #     botao_gold = gr.Button("Transformar Silver → Gold")
    #     gold_tabela = gr.DataFrame(label="Tabela final Gold")
    #     botao_gold.click(transformar_para_gold, outputs=gold_tabela)

demo.launch(debug=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a2fe7b2d45692fef49.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1133, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://a2fe7b2d45692fef49.gradio.live
